<a href="https://colab.research.google.com/github/gretelai/gretel-blueprints/blob/main/docs/notebooks/demo/navigator/reasoning/navigator-data-designer-sdk-reasoning-traces.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🎨 Navigator Data Designer SDK: Synthetic Reasoning Traces

This notebook demonstrates how to use the Gretel Navigator SDK to build a synthetic data generation pipeline tailored for reasoning tasks. Instead of creating multi-turn conversations, we will generate reasoning traces that can be utilized for training and fine-tuning language models with reinforcement learning techniques and invoking chain-of-thought processing.

These synthetic reasoning traces can be used to enhance model performance in areas such as mathematics, coding, scientific reasoning, and other domains that benefit from structured reasoning.

In [1]:
%%capture
# Install the latest version of Gretel client and dependencies
%pip install -U git+https://github.com/gretelai/gretel-python-client

In [2]:
from gretel_client.navigator_client import Gretel
from typing import List
from pydantic import BaseModel, Field

## ⚙️ Data Designer Configuration with the SDK

Instead of relying on a single YAML configuration file, we construct our pipeline interactively. This provides granular control over how we guide LLMs to produce realistic reasoning traces. By adjusting prompts, seed data, and instructions, we can quickly iterate and refine our data generation process.


### 🚀 Initialize Gretel Navigator Data Designer

Instantiate the data designer with the [Gretel API key](https://console.gretel.ai/users/me/key), chosen model suite, and special system instructions.
This initializes the pipeline and ensures that all subsequent synthetic data generation adheres to the defined parameters.

In [3]:
gretel = Gretel(api_key="prompt", endpoint="https://api.dev.gretel.ai")

data_designer = gretel.data_designer.new(
    model_suite="llama-3.x"
)

Found cached Gretel credentials
Logged in as kirit.thadaka@gretel.ai ✅
Gretel client configured to use project: proj_2uY0cfM0kjiegpyEZvCHNKZYxGf


### 🌱 Adding Categorical Seed Columns

Define categorical seed columns that set the context for the generated empathic reasoning traces. For example, domain and theme determine the type of everyday scenario where empathy is crucial, while complexity guides the depth of emotional insight and detailed support.

In [4]:
# Define a domain column that sets the context for empathic scenarios in everyday life.
data_designer.add_column(
    name="domain",
    type="category",
    params={
        "values": [
            "Family Dynamics",
            "Workplace Challenges",
            "Friendship Moments",
            "Community Interactions",
            "Personal Well-being",
            "Unexpected Encounters"
        ]
    }
)

# Add theme subcategories for each domain
data_designer.add_column(
    name="theme",
    type="subcategory",
    params={
        "category": "domain",
        "values": {
            "Family Dynamics": [
                "Parenting Dilemmas",
                "Sibling Rivalries"
            ],
            "Workplace Challenges": [
                "Communication Breakdowns",
                "Leadership Dilemmas"
            ],
            "Friendship Moments": [
                "Support & Understanding",
                "Misunderstandings & Reconciliations"
            ],
            "Community Interactions": [
                "Neighborhood Support",
                "Cultural Celebrations"
            ],
            "Personal Well-being": [
                "Mental Health",
                "Self-care & Reflection"
            ],
            "Unexpected Encounters": [
                "Serendipitous Meetings",
                "Moments of Realization"
            ]
        }
    }
)

# Define a complexity column to guide the level of detail and challenge in the empathic scenarios.
data_designer.add_column(
    name="complexity",
    type="category",
    params={
        "values": ["Basic", "Intermediate", "Advanced"]
    }
)

DataDesigner(
    model_suite: llama-3.x
    sampler_columns: ['domain', 'theme', 'complexity']
)

### ✨ Adding Generated Data Columns

Define the columns that the model will generate. These prompts instruct the LLM to produce the actual empathic reasoning trace and answer, following the specified format with <think> and <answer> tags.

#### Empathic Reasoning Trace Generation

This column is designed to generate clear, thoughtful reasoning traces that blend logical analysis with emotional insight for everyday situations where empathy is crucial. The generation prompt is tailored to:
- Produce a structured explanation that highlights both the practical reasoning and the emotional dynamics at play.
- Encourage a dual output: one part detailing the empathic thought process (enclosed within `<think>` tags) and another delivering a compassionate final answer (enclosed within `<answer>` tags).
- Ensure that the generated content reflects deep understanding, compassion, and a balanced view of the challenges and emotions involved.

In [5]:
special_system_instructions = """
You are an empathic reasoning agent. Your task is to generate realistic and compassionate reasoning traces for common day-to-day situations. Adopt a caring and supportive tone as you provide detailed insights into human experiences and emotions.
- Focus on everyday scenarios where empathy, understanding, and emotional intelligence are key.
- Consider various perspectives, emphasizing the emotional impact of actions and decisions.
- Ensure your reasoning process is clear, structured, and heartfelt, reflecting deep care for the individuals involved.
- Enclose your thoughtful reasoning process within <think>...</think> tags before providing the final JSON output.
"""

data_designer.add_column(
    name="scenario",
    type="llm-text",
    system_prompt=special_system_instructions,
    prompt=(
        "Generate a clear and concise everyday scenario for the {{domain}} domain, theme {{theme}}, and complexity {{complexity}}, "
        "where empathy and understanding play a crucial role. Focus on a situation that highlights emotional challenges or opportunities for compassionate support, and include a specific question or request for help that clearly outlines a problem or challenge needing resolution.\n\n"
        "Guidelines:\n"
        "- Provide only the scenario statement without any additional metadata, solution steps, or internal commentary.\n"
        "- Use everyday language and incorporate realistic, practical context from an empathic perspective.\n"
        "- Ensure the scenario includes a clear follow-up question or request for assistance, making it apparent what the problem or challenge is.\n"
        "- Do not include any formatting tags or markers.\n\n"
        "Examples:\n"
        "1. 'Imagine a situation where a friend is visibly upset after a long, challenging day. What might be causing their distress, and how could you offer support?'\n"
        "2. 'Consider a moment at a family dinner where a subtle conflict arises between members. What could be the underlying issue, and how might empathy help mend the situation?'\n"
        "3. 'Picture a colleague receiving unexpected criticism during a meeting. What are the potential emotional impacts, and what supportive response could be helpful?'\n"
    )
)

DataDesigner(
    model_suite: llama-3.x
    sampler_columns: ['domain', 'theme', 'complexity']
    llm_gen_columns: ['scenario']
)

#### Empathic Reasoning Process Generation

These columns generate and evaluate a detailed empathic reasoning trace for addressing everyday scenarios. The process emphasizes a compassionate, thoughtful approach that blends logical reasoning with emotional insight. The prompts instruct the model to include its internal thought process within <think>...</think> tags before providing the JSON output.

In [6]:
class Thought(BaseModel):
    """A single step in the structured empathic reasoning process.
    This step captures an empathetic observation or insight that informs a thoughtful, compassionate approach to addressing everyday challenges.
    """
    step_number: int = Field(..., ge=1, description="The order of the reasoning step, starting from 1.")
    content: str = Field(..., min_length=5, description="A detailed explanation of this reasoning step, incorporating both logical analysis and emotional insight.")

class ReasoningTrace(BaseModel):
    """A structured empathic reasoning trace for addressing a scenario.
    This model records a step-by-step process that integrates logical analysis with emotional insight and empathy to arrive at a supportive final answer.
    """
    reasoning: List[Thought] = Field(..., description="Step-by-step reasoning leading to the final answer, enriched with empathetic observations and practical insights.")
    answer: str = Field(..., description="The final answer derived from the empathic reasoning process, offering compassionate guidance or resolution.")

class Evaluation(BaseModel):
    """Output format for evaluating an empathic reasoning answer.
    The evaluation assesses the response based on correctness, clarity, and completeness,
    with feedback that emphasizes compassionate insight, clarity, and a holistic understanding of the scenario.
    """
    correctness: float = Field(..., description="Overall correctness rating of the answer (0 to 1).")
    clarity: float = Field(..., description="Clarity rating of the reasoning, including the integration of empathic explanations (0 to 1).")
    completeness: float = Field(..., description="Completeness rating of the reasoning, assessing whether all practical and emotional aspects were considered (0 to 1).")
    feedback: str = Field(..., description="Detailed feedback on the reasoning trace and answer, with suggestions for enhancing empathetic and real-world applicability.")

class FinalEvaluation(Evaluation):
    """Extended evaluation model for final empathic reasoning traces.
    This model adds criteria to assess visual structure and conciseness,
    ensuring the final output is both clear and visually appealing.
    """
    structure: float = Field(...,  description="Rating of the visual structure and formatting (0 to 1), assessing if reasoning steps and final answer are clearly delineated.")
    conciseness: float = Field(..., description="Rating of the conciseness of the reasoning trace (0 to 1), ensuring that extraneous verbosity is minimized.")

In [7]:
data_designer.add_column(
    name="initial_trace",
    type="llm-structured",
    prompt=(
        "You are an empathic reasoning agent. Provide a detailed, step-by-step reasoning process that thoughtfully addresses the following scenario. "
        "Begin by outlining your internal thought process, focusing on both logical considerations and emotional insights, enclosed within <think>...</think> tags. "
        "Then, provide your final compassionate answer.\n\n"
        "Scenario: {scenario}\n\n"
        "Ensure that your response is structured and reflective of a supportive, empathetic approach."
    ),
    output_format=ReasoningTrace
)

DataDesigner(
    model_suite: llama-3.x
    sampler_columns: ['domain', 'theme', 'complexity']
    llm_gen_columns: ['scenario', 'initial_trace']
)

In [8]:
data_designer.add_column(
    name="initial_trace_evaluation",
    type="llm-structured",
    prompt=(
        "<initial_trace>{{initial_trace}}</initial_trace>\n\n"
        "Now, analyze the provided empathic reasoning trace and final answer as if you were an insightful observer assessing both logical and compassionate approaches. "
        "Evaluate the response with a focus on emotional insight, clarity, and holistic consideration.\n\n"
        "Include your internal thought process within <think>...</think> tags before providing the JSON."
    ),
    output_format=Evaluation
)

DataDesigner(
    model_suite: llama-3.x
    sampler_columns: ['domain', 'theme', 'complexity']
    llm_gen_columns: ['scenario', 'initial_trace', 'initial_trace_evaluation']
)

#### Final Empathic Reasoning Trace Generation and Evaluation

These columns refine and evaluate the final empathic reasoning trace. The final trace is generated by reviewing the scenario, your initial empathic reasoning trace, and its evaluation. The process integrates improvements suggested by the evaluation and ensures that the final reasoning is compassionate, clear, and comprehensive. As always, include your internal thought process wrapped within <think>...</think> tags before providing the final JSON output.

In [9]:
data_designer.add_column(
    name="final_trace",
    type="llm-structured",
    prompt=(
        "Review the scenario, your initial empathic reasoning trace, and its evaluation:\n\n"
        "Scenario: {{scenario}}\n\n"
        "Initial Empathic Reasoning Trace:\n{{initial_trace}}\n\n"
        "Initial Trace Evaluation:\n{{initial_trace_evaluation}}\n\n"
        "From the perspective of an empathic reasoning agent, provide a refined final reasoning trace that addresses both the emotional and logical dimensions of the scenario. "
        "Your final trace should be visually structured as follows:\n"
        "1. Present a numbered list of concise reasoning steps. Each step should be clear and free of unnecessary verbosity.\n"
        "2. Include a clearly separated section for the final answer, prefixed with a header (e.g., 'Final Answer:').\n"
        "3. Use visual markers or markdown formatting to enhance readability.\n"
        "Avoid adding extraneous details—focus on clarity and conciseness.\n\n"
        "Also, include your internal thought process wrapped within <think>...</think> tags. "
        "Return only the final, visually structured reasoning trace."
    ),
    output_format=ReasoningTrace
)

data_designer.add_column(
    name="final_trace_evaluation",
    type="llm-structured",
    prompt=(
        "<final_trace>{{final_trace}}</final_trace>\n\n"
        "Analyze the provided empathic reasoning trace and final answer from the viewpoint of an insightful observer. "
        "Evaluate the response focusing on correctness, clarity, and completeness, as well as its visual structure and conciseness. "
        "Assess whether the reasoning steps are clearly separated (e.g., numbered or bullet-pointed) and if the final answer is distinct and succinct.\n\n"
        "Include your internal thought process within <think>...</think> tags before providing the JSON."
    ),
    output_format=FinalEvaluation
)

DataDesigner(
    model_suite: llama-3.x
    sampler_columns: ['domain', 'theme', 'complexity']
    llm_gen_columns: [
        "scenario",
        "initial_trace",
        "initial_trace_evaluation",
        "final_trace",
        "final_trace_evaluation"
    ]
)

## 👀 Generating a dataset preview

- Preview mode allows you to quickly iterate on your data design.

- Each preview generation call creates a sample for inspection, helping you verify prompts and instructions before running a larger batch job.

In [12]:
# Generate a preview
preview = data_designer.preview()

[10:42:13] [INFO] 🚀 Generating preview
[10:42:15] [INFO] 🦜 Step 1: Generate columns using samplers
[10:42:15] [INFO] 🦜 Step 2: Generate column from template v2
[10:42:26] [INFO] 🦜 Step 3: Generate column from template v2 1
[10:43:08] [INFO] 🦜 Step 4: Generate column from template v2 2
[10:43:37] [INFO] 🦜 Step 5: Generate column from template v2 3
[10:44:34] [INFO] 🦜 Step 6: Generate column from template v2 4
[10:45:03] [INFO] 🎉 Your dataset preview is ready!


In [13]:
preview.display_sample_record()

                                                 Generated Columns                                                 
┏━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Name                     ┃ Value                                                                                ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ domain                   │ Workplace Challenges                                                                 │
├──────────────────────────┼──────────────────────────────────────────────────────────────────────────────────────┤
│ theme                    │ Communication Breakdowns                                                             │
├──────────────────────────┼──────────────────────────────────────────────────────────────────────────────────────┤
│ complexity               │ Advanced                                                                             │
├──────────────────────────┼──────────────────────────────────────────────────────────────────────────────────────┤
│ scenario                 │ "Picture a senior team leader, Sarah, who has just been transferred to a new         │
│                          │ department and discovers that her new team has been working under immense pressure   │
│                          │ and stress due to inadequate resources and an overly demanding manager. She notices  │
│                          │ that team members seem hesitant to speak up or share their concerns, and she senses  │
│                          │ a lingering tension in the air. During a team meeting, one of the team members       │
│                          │ breaks down in tears, confessing their struggle to cope with the workload and their  │
│                          │ fear of being judged or penalized for not meeting the manager's expectations. How    │
│                          │ can Sarah, as the new team leader, create a safe space for open communication and    │
│                          │ empathy, and what steps can she take to address the underlying issues and help her   │
│                          │ team build trust and confidence in her?"                                             │
├──────────────────────────┼──────────────────────────────────────────────────────────────────────────────────────┤
│ initial_trace            │ {                                                                                    │
│                          │     'reasoning': [                                                                   │
│                          │         {                                                                            │
│                          │             'step_number': 1,                                                        │
│                          │             'content': "As I consider Sarah's situation, my initial thought is to    │
│                          │ acknowledge the immense pressure and stress the team has been under, which likely    │
│                          │ stems from inadequate resources and an overly demanding manager. This realization    │
│                          │ sparks empathy for the team members who are struggling to cope with their workload   │
│                          │ and fear being judged or penalized. I recognize the importance of creating a safe    │
│                          │ space for open communication, where team members feel comfortable sharing their      │
│                          │ concerns without fear of retribution."                                               │
│                          │         },                                                                           │
│                          │         {                                                                            │
│                          │             'step_number': 

## 🤔 Like what you see?

- Submit a batch workflow!

In [ ]:
workflow_run = data_designer.create(
    num_records=25,
    workflow_run_name="empathic_reasoning_traces",
    wait_for_completion=True
)

In [ ]:
# Check to see if the Workflow is still active.
workflow_run.status

In [ ]:
df = workflow_run.dataset.df
df.head()